In [8]:
!pip install openai tavily-python pypdf python-dotenv

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: C:\Users\ARJUN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
import sys
print(sys.executable)

C:\Users\ARJUN\AppData\Local\Programs\Python\Python310\python.exe


In [10]:
!{sys.executable} -m pip install pypdf

In [11]:
!{sys.executable} -m pip install dotenv


In [12]:
!{sys.executable} -m pip install tavily


In [13]:
import os
from openai import OpenAI
from pypdf import PdfReader
from dotenv import load_dotenv
from tavily import TavilyClient


load_dotenv()


os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_KEY"
os.environ["TAVILY_API_KEY"] = "YOUR_TAVILY_KEY"


In [14]:
client = OpenAI()
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

MODEL = "gpt-4o-mini"

In [15]:
def extract_page_text(pdf_path, page_number):
    reader = PdfReader(pdf_path)

    if page_number < 1 or page_number > len(reader.pages):
        raise ValueError("Invalid page number.")

    return reader.pages[page_number - 1].extract_text()

In [16]:
def planning_step(page_text):

    planning_prompt = """
    You are a planning agent.

    Given a physics textbook page, decide:

    1. Is the page conceptually dense?
    2. Does it contain equations?
    3. Does it likely contain hidden derivations?
    4. What reasoning modes are required?
       - scaling analysis
       - limiting cases
       - derivation expansion
       - conceptual assumption unpacking

    Output structured plan.
    """

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": planning_prompt},
            {"role": "user", "content": page_text}
        ],
        temperature=0.1
    )

    return response.choices[0].message.content

In [24]:
def detect_nuances(page_text, plan):

    prompt = f"""
    You are a nuance detection agent.

    PLAN:
    {plan}

    Identify ONLY nuanced or implicitly reasoned lines.
    Ignore trivial content.

    Output as numbered list with exact quoted text.
    """

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": page_text}
        ],
        temperature=0.2
    )

    return response.choices[0].message.content

In [17]:
def generate_draft(nuances, page_text):

    prompt = f"""
    You are a physics reasoning agent.

    Given these nuanced statements:
    {nuances}

    For each:
    - Expand hidden derivation
    - Show mathematics if required
    - State assumptions
    - Provide limiting cases
    - Explain physical meaning

    Be rigorous.
    """

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": page_text}
        ],
        temperature=0.2
    )

    return response.choices[0].message.content

In [27]:
def verify_explanation(draft):

    verification_prompt = """
    You are a physics verification agent (VIB).

    Evaluate the explanation below.

    Check:
    - Mathematical correctness
    - Logical completeness
    - Missing derivation steps
    - Incorrect assumptions
    - Missing limiting case analysis

    Output:
    1. PASS or FAIL
    2. List weaknesses
    3. Improved version if FAIL
    """

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": verification_prompt},
            {"role": "user", "content": draft}
        ],
        temperature=0.1
    )

    return response.choices[0].message.content

In [19]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "Search the web for physics validation or formula verification.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Physics-related verification query"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

In [38]:
from tavily import TavilyClient
import json

tavily = TavilyClient(api_key=os.getenv("YOUR_TAVILY_KEY"))

def search_web(query):
    

    response = tavily.search(
        query=query,
        search_depth="advanced",
        max_results=5
    )

    combined = ""
    for result in response["results"]:
        combined += f"\nTitle: {result['title']}\nContent: {result['content']}\n"

    return combined

In [33]:
def external_verify(draft):

    messages = [
        {
            "role": "system",
            "content": """
            You are a physics external verification agent.

            If the explanation requires external confirmation,
            call the search_web tool.

            If confident, return final verified explanation.
            """
        },
        {
            "role": "user",
            "content": draft
        }
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )

    message = response.choices[0].message

    # If tool was called
    if message.tool_calls:

        print("\n🌐 Web tool WAS called.\n")

        tool_call = message.tool_calls[0]
        args = json.loads(tool_call.function.arguments)

        web_results = search_web(**args)

        # 🔥 VERY IMPORTANT:
        # Append assistant message with tool_calls
        messages.append(message)

        # Then append tool response
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": web_results
        })

        # Now ask model to finalize
        final_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )

        return final_response.choices[0].message.content

    else:
        print("\n❌ Web tool was NOT used.\n")
        return message.content

In [28]:
def agentic_analysis(page_text):

    print("Step 1: Planning...\n")
    plan = planning_step(page_text)
    print(plan)

    print("\nStep 2: Detecting nuances...\n")
    nuances = detect_nuances(page_text, plan)
    print(nuances)

    print("\nStep 3: Generating draft...\n")
    draft = generate_draft(nuances, page_text)
    print(draft)

    print("\nStep 4: Internal verification...\n")
    verification = verify_explanation(draft)
    print(verification)

    # If FAIL, extract revised version
    if "FAIL" in verification:
        print("\nRefining based on internal verifier...\n")
        draft = verification.split("REVISED_VERSION:")[-1]

    print("\nStep 5: External verification (web enabled)...\n")
    final_output = external_verify(draft)

    return final_output

In [37]:
pdf_path = "constants.pdf"
page_number = int(input("Enter page number: "))

page_text = extract_page_text(pdf_path, page_number)

result = agentic_analysis(page_text)

print("\n\n===== FINAL OUTPUT =====\n")
print(result)

Enter page number:  3


Step 1: Planning...

### Structured Plan

1. **Conceptual Density Assessment**
   - **Is the page conceptually dense?** 
     - Yes, the page discusses complex concepts such as gauge invariance, the Higgs mechanism, spontaneous symmetry breaking, and the role of fundamental constants in the Standard Model. It requires a solid understanding of particle physics and quantum field theory.

2. **Equation Presence**
   - **Does it contain equations?**
     - Yes, the page contains several equations, including those related to the vacuum expectation value (v), the electroweak mixing angle (Θw), and relationships involving the masses of gauge bosons.

3. **Hidden Derivations**
   - **Does it likely contain hidden derivations?**
     - Yes, the text implies derivations related to the relationships between the electroweak mixing angle, gauge boson masses, and fundamental constants, although these derivations are not explicitly shown.

4. **Required Reasoning Modes**
   - **What reasoning modes a